### Webscrapping code for apartments.com

The first code block is the packages and supporting functions for regex parsing.

The second code block is the web scrapping that goes through all the colleges, and pulls out the html, then sends it to be web scraped.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Trying to account for the bottom prices

In [2]:
def text_process(text):
    if text != "Call for Rent" and len(text) > 1: 
        text = text.replace('$', '').replace(',', '')
        if "-" in text:
            array = text.split(" - ")
            return (text_process(array[0]) + text_process(array[1]))/2
        elif "/" in text:
            array = text.split("/")
            return int(array[0])
        else:
            return int(text)

In [3]:
## https://www.youtube.com/watch?v=SPM1tm2ZdK4&t=855s

college_file_list = [ ]
# college_file_list = ['jacksonville-al', 'auburn-alabama', 'ames-ia', 'college-station-tx', 'bloomsburg-pa', 'Ashland-OR']

with open('Collegetowns2.csv', 'r') as file:
    # Create a CSV reader object using DictReader
    for line in file:
        city = line[:-1].replace(',', '-').replace(' ', '-')
        college_file_list.append(city)
college_file_list = college_file_list[1:217]


## To create the granular dataset
row_data = [0] * len(college_file_list)
# Repeat the row data 100 times
data = [row_data] * 100

# Construct DataFrame with column names from college_file_list
college_prices = pd.DataFrame(data, columns=college_file_list)

## Summary statistics

# # Going to append this so that all the data stays in order
# college_prices = pd.DataFrame(college_file_list, columns=["College"])
# college_prices['Mean'] = None
# college_prices['Median'] = None
# college_prices['Min'] = None
# college_prices['Max'] = None

options=Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options )

i = 1
print("Firing up for loop")
for city in college_file_list:
    url = "https://www.apartments.com/" + city + "/max-1-bedrooms/"
    print(url)
    driver.get(url)
    html = driver.page_source
    
    if re.search("404 Page Not Found", html):
        print("Nothing found!")
    else:
        i = 0
        if re.search("property-pricing", html):
            propertyPrice = driver.find_elements(By.CLASS_NAME, "property-pricing")
            for element in propertyPrice:
                value = text_process(element.text)
                if value is not None:
                    college_prices[city].iloc[i] = value
                    i=i+1

        if re.search("property-rents", html):
            propertyrent = driver.find_elements(By.CLASS_NAME, "property-rents")
            for element in propertyrent:
                value = text_process(element.text)
                if value is not None:
                    college_prices[city].iloc[i] = value
                    i=i+1
    

driver.close()

Firing up for loop
https://www.apartments.com/Auburn-AL/max-1-bedrooms/
https://www.apartments.com/Jacksonville-AL/max-1-bedrooms/
https://www.apartments.com/Livingston-AL/max-1-bedrooms/
https://www.apartments.com/Montevallo-AL/max-1-bedrooms/
https://www.apartments.com/Troy-AL/max-1-bedrooms/
https://www.apartments.com/Tuscaloosa-AL/max-1-bedrooms/
https://www.apartments.com/Tuskegee-AL/max-1-bedrooms/
https://www.apartments.com/Fairbanks-AK/max-1-bedrooms/
https://www.apartments.com/Flagstaff-AZ/max-1-bedrooms/
https://www.apartments.com/Arkadelphia-AR/max-1-bedrooms/
https://www.apartments.com/Conway-AR/max-1-bedrooms/
https://www.apartments.com/Fayetteville-AR/max-1-bedrooms/
https://www.apartments.com/Magnolia-AR/max-1-bedrooms/
https://www.apartments.com/Monticello-AR/max-1-bedrooms/
https://www.apartments.com/Russellville-AR/max-1-bedrooms/
https://www.apartments.com/Arcata-CA/max-1-bedrooms/
https://www.apartments.com/Chico-CA/max-1-bedrooms/
https://www.apartments.com/Davis-C

https://www.apartments.com/Stillwater-OK/max-1-bedrooms/
https://www.apartments.com/Tahlequah-OK/max-1-bedrooms/
https://www.apartments.com/Weatherford-OK/max-1-bedrooms/
https://www.apartments.com/Ashland-OR/max-1-bedrooms/
https://www.apartments.com/Corvallis-OR/max-1-bedrooms/
https://www.apartments.com/Monmouth-OR/max-1-bedrooms/
https://www.apartments.com/Bloomsburg-PA/max-1-bedrooms/
https://www.apartments.com/California-PA/max-1-bedrooms/
https://www.apartments.com/Clarion-PA/max-1-bedrooms/
https://www.apartments.com/East-Stroudsburg-PA/max-1-bedrooms/
https://www.apartments.com/Edinboro-PA/max-1-bedrooms/
https://www.apartments.com/Indiana-PA/max-1-bedrooms/
https://www.apartments.com/Kutztown-PA/max-1-bedrooms/
https://www.apartments.com/Lock-Haven-PA/max-1-bedrooms/
https://www.apartments.com/Mansfield-PA/max-1-bedrooms/
https://www.apartments.com/Millersville-PA/max-1-bedrooms/
https://www.apartments.com/Shippensburg-PA/max-1-bedrooms/
https://www.apartments.com/Slippery-Ro

In [4]:
college_prices

,Auburn-AL,Jacksonville-AL,Livingston-AL,Montevallo-AL,Troy-AL,Tuscaloosa-AL,Tuskegee-AL,Fairbanks-AK,Flagstaff-AZ,Arkadelphia-AR,...,La-Crosse-WI,Madison-WI,Menomonie-WI,Platteville-WI,River-Falls-WI,Stevens-Point-WI,Whitewater-WI,Laramie-WY,-,-
0,1400.0,459.0,0,1349.0,662.5,2038.5,905.0,1364,1836.0,805,...,1027.5,1800.0,1325,907.5,1425,1225.0,1100.0,800,0,0
1,975.0,775.0,0,1007.0,811.0,1217.5,635.0,1400,4198.5,0,...,855.0,1445.0,1395,649.0,1440,1349.0,1145.0,0,0,0
2,1137.5,892.5,0,1055.0,667.5,852.5,482.5,1777,2443.0,0,...,1475.0,1195.0,754,752.5,995,937.5,840.0,0,0,0
3,962.5,824.0,0,1156.0,0.0,1587.0,1462.5,1298,2057.5,0,...,915.0,1027.5,0,850.0,1252,1320.0,999.0,0,0,0
4,965.0,895.0,0,1201.0,0.0,950.0,1174.5,0,2455.0,0,...,1497.5,2300.0,0,777.5,950,857.5,1532.5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0,0.0,0.0,0,0,0
96,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0,0.0,0.0,0,0,0
97,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0,0.0,0.0,0,0,0
98,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,...,0.0,0.0,0,0.0,0,0.0,0.0,0,0,0


In [5]:
college_prices.to_csv('collegePrices_granular.csv', index=False)

### Code repository from times of old


In [6]:

# # Custom function to extract value after dollar sign
# def extract_value_after_dollar(row):
#     # Find the index of the dollar sign
#     dollar_index = row.find('$')
#     # Extract everything after the dollar sign
#     value_after_dollar = row[dollar_index + 1:]
#     return value_after_dollar

# def extract_average(html): 
#     if re.search("404 Page Not Found", html):
#         return None
#     else: 
#         pattern = r'aria-label="[^"]*">[^<]*<p class="property-pricing">[^<]*</p>'
#         prices = re.findall(pattern, html)

#         df = pd.DataFrame(prices, columns=['Text'])
        
#         if len(df) < 5:
#             return None

#         # Separate the 'Text' column into 'AptName' and 'Price' columns
#         df[['AptName', 'Price']] = df['Text'].str.split('>[^<]*<p', expand=True)

#         # Clean 'AptName' and 'Price' columns
#         df['AptName'] = df['AptName'].str.replace('aria-label="', '')
#         df['AptName'] = df['AptName'].str.replace(r',[^,]*$', '')
#         df['Price'] = df['Price'].str.replace('class="property-pricing">\\$', '').str.replace('</p>', '')

#         # Separate 'Price' column into 'Price1' and 'Price2' columns
#         df[['Price1', 'Price2']] = df['Price'].str.split(' - ', expand=True)

#         # Drop the 'Text' and 'Price' columns
#         df.drop(columns=['Text', 'Price'], inplace=True)
#         df['Price2'] = pd.to_numeric(df['Price2'].str.replace(',', ''), errors='coerce')
#         df['Price1'] = df['Price1'].apply(extract_value_after_dollar)
#         df['Price1'] = pd.to_numeric(df['Price1'].str.replace(',', ''), errors='coerce')

#         return((df['Price1'].mean() + df['Price2'].mean())/2)
# #     return df

# ## https://www.youtube.com/watch?v=SPM1tm2ZdK4&t=855s

# college_file_list = []
# # college_file_list = ['jacksonville-al', 'auburn-alabama', 'ames-ia', 'college-station-tx']

# with open('Collegetowns2.csv', 'r') as file:
#     # Create a CSV reader object using DictReader
#     for line in file:
#         city = line[:-1].replace(',', '-').replace(' ', '-')
#         college_file_list.append(city)
# college_file_list = college_file_list[1:]

# # Going to append this so that all the data stays in order
# college_prices = pd.DataFrame(college_file_list, columns=["College"])
# college_prices['Mean'] = None
# college_prices['Median'] = None
# college_prices['Min'] = None
# college_prices['Max'] = None

# options=Options()
# options.add_experimental_option("detach", True)

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
#                           options=options )

# i = 1
# print("Firing up for loop")
# for city in college_file_list:
#     url = "https://www.apartments.com/" + city + "/max-1-bedrooms/"
# #     print(url)
#     driver.get(url)

#     html = driver.page_source
#     time.sleep(2)
#     town_avg = extract_average(html)
#     college_prices.loc[college_prices['College'] == city, 'Mean'] = town_avg
#     i= i+1
#     time.sleep(1)
#     if i > 215:
#         break

# driver.close()